In [23]:
import pandas as pd
import pickle
import os
from os.path import join
import numpy as np

import torch
from torch import cuda
from torch.optim import Adam

import torchkge
from torchkge.models import Model, TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader

from tqdm.autonotebook import tqdm

## pathnames-adarshm
wot_path = '../'
svo_data_path = join(wot_path, 'data/SVO-tensor-dataset/80')

In [24]:
svo_data_path

'../data/SVO-tensor-dataset/80'

In [25]:
model_path = join(wot_path, 'models')

os.makedirs(model_path, exist_ok=True)

files = os.listdir(svo_data_path)
for f in files:
    if 'train' in f: tr_fp = f
    if 'valid' in f: val_fp = f
    if 'test' in f: test_fp = f

def read_data(tr_fp, val_fp, test_fp):
    tr_df = pd.read_csv(join(svo_data_path, tr_fp),
                       sep='\t', header=None, names=['from', 'rel', 'to'])
    val_df = pd.read_csv(join(svo_data_path, val_fp),
                       sep='\t', header=None, names=['from', 'rel', 'to'])
    test_df = pd.read_csv(join(svo_data_path, test_fp),
                       sep='\t', header=None, names=['from', 'rel', 'to'])
    return tr_df, val_df, test_df

In [26]:
tr_fp

'svo_data_ts80_train_1000000.dat'

In [27]:
files

['svo_data_ts80_test_50000.dat',
 'svo_data_ts80_valid_250000.dat',
 'svo_data_ts80_train_1000000.dat']

In [28]:
class CustomTransModel(torchkge.models.interfaces.TranslationModel):
    def __init__(self, kg, model_type, **kwargs):
        self.kg = kg
        self.diss_type = kwargs.pop('diss_type', 'L2')
        super().__init__(self.kg.n_ent, self.kg.n_rel, self.diss_type)

        if model_type in ['TransR', 'TransD', 'TorusE']:
            self.ent_emb_dim = kwargs.pop('ent_emb_dim', 250)
            self.rel_emb_dim = kwargs.pop('rel_emb_dim', 250)
            self.model = getattr(torchkge.models, model_type + 'Model')(self.ent_emb_dim,
                                    self.rel_emb_dim, n_entities = self.kg.n_ent, n_relations = self.kg.n_rel)
        else:
            self.emb_dim = kwargs.pop('emb_dim', 250)
            self.model = getattr(torchkge.models, model_type + 'Model')(self.emb_dim, n_entities = self.kg.n_ent, 
                                    n_relations = self.kg.n_rel)
        
        ## Hyperparameters
        self.lr = kwargs.pop('lr', 0.0004)
        self.n_epochs = kwargs.pop('n_epochs', 100)
        self.b_size = kwargs.pop('b_size', 32)
        
        # Legacy code
        # super(CustomTransModel, self).__init__(self.emb_dim, kg.n_ent, kg.n_rel,
        #                     dissimilarity_type=self.diss_type)
        

        try:
            self.dataloader = DataLoader(self.kg, batch_size=self.b_size, use_cuda='all')
        except AssertionError:
            self.dataloader = DataLoader(self.kg, batch_size=self.b_size)

        ## Logger
        self.epochs=0
        self.tr_losses=[]
        self.best_epoch=-1
        self.val_losses=[]

    def set_optimizer(self, optClass=Adam, **kwargs):
        self.optimizer = optClass(self.model.parameters(), lr=self.lr,
                                    **kwargs)

    def set_sampler(self, samplerClass=BernoulliNegativeSampler, **kwargs):
        self.sampler = samplerClass(**kwargs)

    def set_loss(self, lossClass=MarginLoss, **kwargs):
        self.loss_fn = lossClass(**kwargs)
        try:
            self.loss_fn.cuda()
        except:
            pass

    def one_epoch(self):
        running_loss = 0.0
        for i, batch in enumerate(self.dataloader):
            h, t, r = batch
            n_h, n_t = self.sampler.corrupt_batch(h, t, r)
            self.optimizer.zero_grad()
            pos, neg = self.model(h, t, n_h, n_t, r)
            loss = self.loss_fn(pos, neg)
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()
        self.model.normalize_parameters()
        self.epochs += 1
        epoch_loss = running_loss/i
        self.tr_losses.append(epoch_loss)
        return epoch_loss

    def validate(self, val_kg):
        losses = []
        for batch in DataLoader(val_kg, batch_size=self.b_size):
            h, t, r = batch
            n_h, n_t = self.sampler.corrupt_batch(h, t, r)
            pos, neg = self.model(h, t, n_h, n_t, r)
            loss = self.loss_fn(pos, neg)
            losses.append(loss.item())
        return np.mean(losses)

    def train_model(self, n_epochs, val_kg):
        self.model.normalize_parameters()
        epochs = tqdm(range(n_epochs), unit='epoch')
        for epoch in epochs:
            mean_epoch_loss = self.one_epoch()
            if (epoch+1%100)==0 or epoch==0:
                torch.save(self.model.state_dict(), join(model_path,
                                                    'transe_model.pt'))
                val_loss = self.validate(val_kg)
                if not self.val_losses or val_loss < min(self.val_losses):
                    self.best_epoch = epoch
                    torch.save(self.model.state_dict(), join(model_path, 
                                'best_transe_model.pt'))
            epochs.set_description(
                'Epoch {} | mean loss: {:.5f}'.format(epoch + 1, mean_epoch_loss))



In [29]:
tr_df, val_df, test_df = read_data(tr_fp, val_fp, test_fp)
sizes = [df.shape[0] for df in (tr_df, val_df, test_df)]
full_df = pd.concat([tr_df, val_df, test_df])

In [30]:
tr_df.shape

(1000000, 3)

In [31]:
full_kg = torchkge.data_structures.KnowledgeGraph(full_df)
tr_kg, val_kg, test_kg = full_kg.split_kg(sizes=sizes)

In [32]:
full_kg.n_rel

4538

In [33]:
tr_kg.n_rel, tr_kg.n_ent

(4538, 30492)

In [22]:
te_mod = CustomTransModel(tr_kg, model_type = 'TransE')
te_mod.set_sampler(samplerClass=BernoulliNegativeSampler, kg=tr_kg)
te_mod.set_optimizer(optClass=Adam)
te_mod.set_loss(lossClass=MarginLoss, margin=0.5)

In [24]:
# Move everything to CUDA if available
if cuda.is_available():
    print("Using cuda.")
    cuda.empty_cache()
    te_mod.model.cuda()
    te_mod.loss_fn.cuda()

Using cuda.


In [25]:
te_mod

CustomTransModel(
  (model): TransEModel(
    (ent_emb): Embedding(30492, 250)
    (rel_emb): Embedding(4538, 250)
  )
  (loss_fn): MarginLoss(
    (loss): MarginRankingLoss()
  )
)

In [26]:
te_mod.train_model(2, val_kg)

0%|          | 0/2 [22:34<?, ?epoch/s]


RuntimeError: Expected object of device type cuda but got device type cpu for argument #3 'index' in call to _th_index_select